# Grayscale 3D CNN

In [1]:
%%python3 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable


In [2]:
#%pip install tensorflow_docs --upgrade
%pip install mediapipe=='0.10.9'
%pip install opencv-python=='4.8.0.76'

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
from utils import VideoDataset, validate_model, train_model, display_frames, display_learning_dynamic, classification_model_metrics

2024-01-20 19:52:02.998804: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-20 19:52:04.395055: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-20 19:52:07.775748: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import cv2
cv2.__version__

'4.8.0'

In [5]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn

In [6]:
use_cuda = True
seed = 1

train_batch_size = 1
test_batch_size = 1
val_batch_size = 1
IMG_SIZE = 112

limit_list = []

min_frame_count = 25


annotations_file = "/home/jupyter/mnt/s3/rsl-videos/slovo/slovo_annotations/SLOVO_DATAFRAME.tsv" # "/content/drive/MyDrive/slovo/SLOVO_DATAFRAME.tsv" #
video_dir = "/home/jupyter/mnt/s3/rsl-videos/slovo/slovo" #"/content/drive/MyDrive/slovo/slovo" #

face_detection_model = 'face_detection_yunet_2023mar.onnx'


In [7]:
video_labels = pd.read_csv(annotations_file, sep='\t')
video_labels['group_rank'] = video_labels.groupby(['text']).cumcount()+1;
video_labels['dataset'] = np.where(video_labels['group_rank']<17,'train', np.where(video_labels['group_rank']<19,'val', 'test'))
if len(limit_list)>0:
  video_labels = video_labels[video_labels.text.isin(limit_list)]
video_labels.tail(5)

,attachment_id,text,user_id,height,width,length,begin,end,group_rank,dataset
19995,81cab3c3-80be-4eb1-b3bf-fd1b69034a16,отложенный,4d26ad833542291c50de9ce88fb4f64a,640,360,131.0,12,73,20,test
19996,4eeeea02-df4c-4667-872e-8387ed31c32e,башенные часы,3018b64d2c938f5b6a0826dfdf486f2c,640,360,145.0,17,83,20,test
19997,d947ab2e-0b6e-4ea7-a5dd-b401c572d177,дважды,3018b64d2c938f5b6a0826dfdf486f2c,640,360,127.0,13,63,20,test
19998,03edb6f1-1c91-46ce-8394-b74bca3b3cd7,время от 0 ночи до 12 дня,472fb939766ece63eb06e2d0e10eb679,840,358,178.0,16,138,19,test
19999,5393ec0c-85a4-4980-a2da-e84259050d83,время от 0 ночи до 12 дня,30eae85220a3d47b79b9955a08d31d63,640,360,205.0,32,172,20,test


In [8]:
classes = tuple(video_labels.text.unique())
n_class = len(classes)
print(f"Total number of classes: {n_class}. Example classes: {classes[:5]}")

Total number of classes: 1000. Example classes: ('Ё', 'А', 'Р', 'Е', 'Ч')


In [9]:
classes.index('слон')

104

In [10]:
train_kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}
test_kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}
val_kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}

In [ ]:
test_dataloader = DataLoader(VideoDataset(video_labels, video_dir, IMG_SIZE, min_frame_count, classes, face_detection_model, ds_type = "test"), batch_size=test_batch_size, shuffle=True, **test_kwargs)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
torch.save(test_dataloader, '/home/jupyter/mnt/s3/rsl-videos/slovo/color_frames_test_dataloader.pth')

In [ ]:
val_dataloader = DataLoader(VideoDataset(video_labels, video_dir, IMG_SIZE, min_frame_count, classes, face_detection_model, ds_type = "val"), batch_size=val_batch_size, shuffle=True, **val_kwargs)

In [ ]:
torch.save(val_dataloader, '/home/jupyter/mnt/s3/rsl-videos/slovo/color_frames_val_dataloader.pth')

In [ ]:
train_dataloader = DataLoader(VideoDataset(video_labels, video_dir, IMG_SIZE, min_frame_count, classes, face_detection_model, ds_type = "train"), batch_size=train_batch_size, shuffle=True, **train_kwargs)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [ ]:
#torch.save(train_dataloader, '/home/jupyter/mnt/s3/rsl-videos/slovo/color_frames_train_dataloader.pth')
torch.save(train_dataloader, 'color_frames_train_dataloader.pth')